This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# # # # # 
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Reference Documentation: 
# Getting Started: https://blog.quandl.com/getting-started-with-the-quandl-api
# Parameter Reference: https://docs.quandl.com/docs/parameters-2#section-times-series-parameters

# url ends with a query (?)
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?'
start_date = 'start_date=2012-08-08'
end_date = 'end_date=2012-08-11'
ascend = 'order=asc'
descend = 'order=desc'
api_key = 'api_key=' + API_KEY

r = requests.get(url + start_date + '&' + end_date + '&' + ascend + '&' + api_key)

# Header information
header_data = r.headers

# # # Practice: Print out all the column names in new line
# for i in header_file.keys():
#     print(i)

# Store in json object 
json_data = r.json()
    
# Sanity check for type     
print(type(header_data))
print(type(json_data))

# Make sure the below is equal (two different approaches for the same result)
r.headers.get('date') == header_data['date']


<class 'requests.structures.CaseInsensitiveDict'>
<class 'dict'>


True

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

sample_col_list = json_data['dataset']['column_names']
sample_data_list = json_data['dataset']['data']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### ANSWERS

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?'
start_date = 'start_date=2017-01-01'
end_date = 'end_date=2017-12-31'
ascend = 'order=asc'
descend = 'order=desc'
api_key = 'api_key=' + API_KEY

r = requests.get(url + start_date + '&' + end_date + '&' + ascend + '&' + api_key)

# check status_code to verfity that requests was successful
r.status_code


200

2. Convert the returned JSON object into a Python dictionary.

In [6]:
json_data = r.json()
print(type(json_data))

<class 'dict'>


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
col_list = json_data['dataset']['column_names']
data_list = json_data['dataset']['data']

# Sanity check that data entries are equal to the # of columns
assert len(col_list) == len(data_list[0])

# 'Open' Prices are stored in the second column (index: 1)
# print(col_list[1])

# Create a list of opening prices using list comprehension 
# Will be iterating over len(data_list)
# Make sure to include conditional to drop 'None' values since sorted() can't compare 'None' to 'float'

opening_price_list = [data_list[i][1] for i in range(len(data_list)) if data_list[i][1] != None]
opening_price_list = sorted(opening_price_list)

lowest = opening_price_list[0]
highest = opening_price_list[-1]

statement = "During the period of 2017-01-01 and 2017-12-31, the lowest opening price \
was ${}. The highest opening price was ${}."
print(statement.format(lowest, highest))

During the period of 2017-01-01 and 2017-12-31, the lowest opening price was $34.0. The highest opening price was $53.11.


4. What was the largest change in any one day (based on High and Low price)?

In [8]:
# Index 2 is High / Index 3 is Low 
col_list[2]
col_list[3] 

# High - Low 

price_change_list = [(data_list[i][2] - data_list[i][3]) for i in range(len(data_list)) if ((data_list[i][2] != None) & (data_list[i][3] != None))]

price_change_list = sorted(price_change_list, reverse=True) # descending order
largest = price_change_list[0] 
# print(type(largest))

statement = "Largest change in any one day (based on High and Low price) is ${0:.2f}"

print(statement.format(largest))


Largest change in any one day (based on High and Low price) is $2.81


5. What was the largest change between any two days (based on Closing Price)?

In [9]:
# print(col_list)

# Index 4 is Close Price 
col_list[4]

# between any two days means consecutive days 
# I took the absolute value of the difference 

closing_price_change = [abs(data_list[i][4] - data_list[i+1][4]) for i in range(len(data_list) - 1) if (data_list[i][4] != None)]

closing_price_change = sorted(closing_price_change, reverse=True)
change = closing_price_change[0]

statement = "Largest change between any two days (based on Closing Price) is ${0:.2f}"

print(statement.format(change))


Largest change between any two days (based on Closing Price) is $2.56


6. What was the average daily trading volume during this year?

In [10]:
# print(col_list)

# Index 6 is Traded Volume
col_list[6]

volume = [data_list[i][6] for i in range(len(data_list)) if (data_list[i][6] != None)]
average_volume = sum(volume)/len(volume)  # since we're not using any third-party packages

statement = "Average Daily Volume during the year 2017 is {0:.3f}"

print(statement.format(average_volume))
 

Average Daily Volume during the year 2017 is 89124.337


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [11]:
volume = [data_list[i][6] for i in range(len(data_list)) if (data_list[i][6] != None)]

def median_of_list(a_list):
    
    # important to sort the values first to get the median! 
    a_list = sorted(a_list)
    length = len(a_list)
    
    if (length % 2 == 1):
        idx = (length // 2)     # use integer division (ex. 5//2 = 2)
                                # also remember that indexing starts from zero! 
        median = a_list[idx]    # (if indexing started from 1, would have added 1 to idx)
        

    elif (length % 2 == 0):
        idx = int(length / 2)       # normal division and cast it as int
        median = (a_list[idx - 1] + a_list[idx]) / 2

    return median

median = median_of_list(volume)
statement = "Median trading volume during the year of 2017 was {0:.3f}"

print(statement.format(median))


Median trading volume during the year of 2017 was 76286.000
